In [ ]:
import os, sys, re
import pandas as pd
import numpy as np

def probeix(df, vmin, vmax):
    return np.logical_and((df >= vmin).all(axis = 1), (df <= vmax).all(axis = 1))

def get_wrong_measurements(df, idx, probe, desc, th = 0.5):
#     desc2 = desc.sort_index()
    x = idx[idx == False].index.droplevel(6).drop_duplicates()
    d = []
    s = [((idx.xs(t, level = x.names) == True).sum() / idx.xs(t, level = x.names).shape[0]) for t in x]
    out = desc.loc[x[np.array(s) < th]].query("probe == '%s'" % (probe))
    return out, s

In [ ]:
datapath = '/home/nuburu/sandbox/data/characterization/autoscan/autoscan.h5'
savepath = '/home/nuburu/Documents/'

# load the data
df = pd.read_hdf(datapath, key = 'data')
desc = pd.read_hdf(datapath, key = 'description')

velcols = df.columns[-4:-2]
permcol = 'perm'
hammcol = 'e_star'

In [ ]:
# check velocities
vels = df.loc[:, velcols]
## firtst check how they are distributed
vels.describe().loc[['mean', 'min', 'max'], :]

In [ ]:
## identify offending values and those to keep
velmin = 0.5e3
velmax = 7.0e3
velidx = probeix(vels, vmin = velmin, vmax = velmax)

## check the data makes sense
print(vels.loc[velidx, :].describe())

## print the labels that have problem
out, _ = get_wrong_measurements(vels, velidx, 'vel', desc)
out.sample(10)

In [ ]:
perm = df.loc[:, [permcol]]
print(perm.describe())
permidx = probeix(perm, vmin = 0.3, vmax = 5e3)
## check the data makes sense
print(perm.loc[permidx, :].describe())

## print the labels that have problem
out, _  = get_wrong_measurements(perm, permidx, 'perm', desc)
out.sample(10)

In [ ]:
# remove offending values and keep just good measurements
idx = np.logical_and(velidx.values, permidx.values)
dc = df.loc[idx, :]

In [ ]:
# alternative, fill the values with means or nans
vels.loc[velidx == False, :] = np.nan
desc.loc[vels.index[velidx == False].droplevel(6).drop_duplicates()].query("probe == 'vel'")

In [ ]:
# x = velidx[velidx == False].index.droplevel(6).drop_duplicates()
# for t in x:
#     vels.loc[mix[[*t], :], :]

In [ ]:
# means = vels.groupby(level = velidx.index.names[:-1], sort = False).apply(np.mean)

# for dd in means.index:
#     pass
# dd

In [ ]:
# mix = pd.IndexSlice
# mix[[*ix], :]
# vels.loc(axis = 0)[mix[[dd], :], :]

In [ ]:
# def myquery(x, vels, velmin = 0.5e3, velmax = v):
#     v = vels.copy()
#     s = v.loc[x, :].shape[0]
#     idx = np.logical_and((v >= velmin).all(axis = 1), (v <= velmax).all(axis = 1))
#     skeep = np.sum(idx)
#     sdrop = np.sum(idx == False)
#     return s, skeep, sdrop

# for a, b in idx.loc[revise_idx, :].groupby(level = idx.index.names[2:-1]):
#     pass#.describe()

# x = revise_idx.droplevel(6).drop_duplicates()
# idx.loc[slice(x, :), :]

# idx.loc[revise_idx, :]
# idx.groupby(level = idx.index.names[2:-1]).describe()
# .loc[:, 'r'] = 'review'

# mix = pd.IndexSlice
# pd.concat((idx.loc[mix[[*t], :]] for t in x)) 

# mix = pd.IndexSlice
# pd.concat((idx.loc[mix[[*t], :]] for t in x)) 